# 2022/09/02

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.utils.prune as prune
import torch.nn.functional as F

from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split

from scipy import io
import os

In [ ]:
import wandb
# wandb.init(project="test")

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
sweep_config = {
    'method': 'grid'
    }

parameters_dict = {
    'hidden_size': {
        'values': [2, 8, 16]
        },
    'batch_size': {
        'values': [64,128]
        },
    'optimizer': {
        'values': ['adam']
        },
    'epochs': {
        'values': [500]
        },
    'learning_rate': {
        'values': [0.01]
      },
    'subject_label': {
        'values': [1, 2,3,4,5,6,7,8,9]
      },
    'chop': {
        'values': [16,32,64]
      },  
    }

sweep_config['parameters'] = parameters_dict

In [ ]:
sweep_id = wandb.sweep(sweep_config, project="Calib_data_Aug_5_10_15")
config = wandb.config

### MAT file에서 data 읽고 준비

In [ ]:
def load_mat_file_train(num_subject, chop, k_group, a_group, y_group):
    # mat_file = io.loadmat('/Users/goldenyoo/Library/Mobile Documents/com~apple~CloudDocs/BioCAS_prepare/Python_code/Data_center/one_dx/Calib_data_'+ str(num_subject) +'.mat')
    mat_file = io.loadmat('C:/Users/Peter/iCloudDrive/BioCAS_prepare/BCIIV_2a_mat/myData/Aug/Calib_data_'+ str(num_subject) +'_chop_'+str(chop) +'.mat')
    

    K1 = mat_file['K1']
    A1 = mat_file['A1']

    K2 = mat_file['K2']
    A2 = mat_file['A2']

    Y1 = mat_file['Y1']
    Y2 = mat_file['Y2']

    # K 특성에 대한 Class1 vs Class2 Data 가져오기
    k1 = torch.FloatTensor(K1)
    k1 = k1.transpose(0,2)

    k2 = torch.FloatTensor(K2)
    k2 = k2.transpose(0,2)

    # A 특성에 대한 Class1 vs Class2 Data 가져오기
    a1 = torch.FloatTensor(A1)
    a1 = a1.transpose(0,2)

    a2 = torch.FloatTensor(A2)
    a2 = a2.transpose(0,2)

    # Y에 대한 Class1 vs Class2 Data 가져오기
    y1 = torch.LongTensor(Y1)
    y2 = torch.LongTensor(Y2)

    k_train = torch.cat([k1,k2],dim=0)
    a_train = torch.cat([a1,a2],dim=0)

    y_train = torch.cat([y1,y2],dim=0)
    y_train = y_train-1 # y를 0~1의 정수로 만들어야함.

    #group 만들기
    if num_subject == 1:
        k_group = k_train.to(device)
        a_group = a_train.to(device)
        y_group = y_train.to(device)
    else:
        k_group = torch.cat([k_group.to(device),k_train.to(device)], dim=0).to(device)
        a_group = torch.cat([a_group.to(device),a_train.to(device)], dim=0).to(device)
        y_group = torch.cat([y_group.to(device),y_train.to(device)], dim=0).to(device)

    return k_group, a_group, y_group


def load_mat_file_test(num_subject, chop, k_group, a_group, y_group):
    # mat_file = io.loadmat('/Users/goldenyoo/Library/Mobile Documents/com~apple~CloudDocs/BioCAS_prepare/Python_code/Data_center/one_dx/Eval_data_'+ str(num_subject) +'.mat')
    mat_file = io.loadmat('C:/Users/Peter/iCloudDrive/BioCAS_prepare/BCIIV_2a_mat/myData/Eval_data_'+ str(num_subject) +'_chop_'+str(chop) +'.mat')

    K1 = mat_file['K1']
    A1 = mat_file['A1']

    K2 = mat_file['K2']
    A2 = mat_file['A2']

    Y1 = mat_file['Y1']
    Y2 = mat_file['Y2']

    # K 특성에 대한 Class1 vs Class2 Data 가져오기
    k1 = torch.FloatTensor(K1)
    k1 = k1.transpose(0,2)

    k2 = torch.FloatTensor(K2)
    k2 = k2.transpose(0,2)

    # A 특성에 대한 Class1 vs Class2 Data 가져오기
    a1 = torch.FloatTensor(A1)
    a1 = a1.transpose(0,2)

    a2 = torch.FloatTensor(A2)
    a2 = a2.transpose(0,2)

    # Y에 대한 Class1 vs Class2 Data 가져오기
    y1 = torch.LongTensor(Y1)
    y2 = torch.LongTensor(Y2)

    k_train = torch.cat([k1,k2],dim=0)
    a_train = torch.cat([a1,a2],dim=0)

    y_train = torch.cat([y1,y2],dim=0)
    y_train = y_train-1 # y를 0~1의 정수로 만들어야함.

    #group 만들기
    if num_subject == 0:
        k_group = k_train.to(device)
        a_group = a_train.to(device)
        y_group = y_train.to(device)
    else:
        k_group = torch.cat([k_group.to(device),k_train.to(device)], dim=0)
        a_group = torch.cat([a_group.to(device),a_train.to(device)], dim=0)
        y_group = torch.cat([y_group.to(device),y_train.to(device)], dim=0)

    return k_group, a_group, y_group
    

In [ ]:
# k_train = []
# a_train = []
# y_train = []

# k_test = []
# a_test = []
# y_test = []

# for i in range(1,10):
#     k_train, a_train, y_train = load_mat_file_train(i, k_train, a_train, y_train)
#     k_test, a_test, y_test = load_mat_file_test(i, k_test, a_test, y_test)

### Dataset & DataLoader

In [ ]:
def build_dataset(batch_size,k_train, a_train, y_train, k_test, a_test, y_test):
    dataset_train = TensorDataset(k_train.to(device)   ,a_train.to(device),   y_train.to(device)) # 각 tensor의 첫번째 dim이 일치해야한다
    dataset_test  = TensorDataset(k_test.to(device)    ,a_test.to(device) ,   y_test.to(device) ) # 각 tensor의 첫번째 dim이 일치해야한다

    # Data Split
    dataset_size = len(dataset_train)
    train_size = int(dataset_size * 0.8)
    validation_size = dataset_size - train_size
    train_dataset, valid_dataset = random_split(dataset_train, [train_size, validation_size])

    train_dataloader    = DataLoader(train_dataset  ,batch_size=      batch_size  , shuffle=True, drop_last=True)
    valid_dataloader    = DataLoader(valid_dataset  ,batch_size= int(batch_size/8), shuffle=True, drop_last=True)

    # Data Split
    test_size = len(dataset_test)
    test_dataloader = DataLoader(dataset_test, batch_size= test_size, shuffle=True, drop_last=True)

    return train_dataloader, valid_dataloader, test_dataloader, test_size

In [ ]:
def build_optimizer(network, optimizer, learning_rate):
    if optimizer == "sgd":
        optimizer = optim.SGD(network.parameters(),
                              lr=learning_rate, momentum=0.9)
    elif optimizer == "adam":
        optimizer = optim.Adam(network.parameters(),
                               lr=learning_rate)
    return optimizer


In [ ]:
input_size = 22
n_class = 2

dtype = torch.float

class TextLSTM(nn.Module):
  def __init__(self,hidden_size):
    super(TextLSTM, self).__init__()

    self.lstm_1 = nn.LSTM(input_size=input_size, hidden_size=hidden_size)
    self.lstm_2 = nn.LSTM(input_size=input_size, hidden_size=hidden_size)
    self.fc = nn.Linear(hidden_size*2, n_class)

  def forward(self, hidden_and_cell_k, hidden_and_cell_a, K_and_A):
    (k, a) = K_and_A

    k = k.transpose(1,2)
    k = k.transpose(0,1)
    a = a.transpose(1,2)
    a = a.transpose(0,1)

    outputs1, (h_n1,c_n1) = self.lstm_1(k, hidden_and_cell_k)
    outputs2, (h_n2,c_n2) = self.lstm_2(a, hidden_and_cell_a)

    outputs = torch.cat((h_n1[-1],h_n2[-1]), dim=1)  

    model = self.fc(outputs)  # 최종 예측 최종 출력 층
    return model.to(device)

In [ ]:
def train(config=None):
    # Initialize a new wandb run
    with wandb.init(config=config):
        # If called by wandb.agent, as below,
        # this config will be set by Sweep Controller
        config = wandb.config


        k_train = torch.tensor([]).to(device)
        a_train = torch.tensor([]).to(device)
        y_train = torch.LongTensor([]).to(device)

        k_test = torch.tensor([]).to(device)
        a_test = torch.tensor([]).to(device)
        y_test = torch.LongTensor([]).to(device)

        k_train, a_train, y_train = load_mat_file_train(config.subject_label, config.chop,k_train, a_train, y_train)
        k_test, a_test, y_test = load_mat_file_test(config.subject_label, config.chop, k_test, a_test, y_test)

        model = TextLSTM(hidden_size=config.hidden_size).to(device)
        criterion = nn.CrossEntropyLoss()
        optimizer = build_optimizer(model, config.optimizer, config.learning_rate)

        model.train()
        
        train_dataloader, valid_dataloader, test_dataloader, test_size = build_dataset(config.batch_size, k_train.to(device), a_train.to(device), y_train.to(device), k_test.to(device), a_test.to(device), y_test.to(device))

        for epoch in range(config.epochs):
            for batch_idx, samples in enumerate(train_dataloader):

                k_train_mb, a_train_mb, y_train_mb = samples

                hidden_k  = torch.zeros(1, config.batch_size, config.hidden_size, requires_grad=True).to(device)
                cell_k    = torch.zeros(1, config.batch_size, config.hidden_size, requires_grad=True).to(device)
                hidden_a  = torch.zeros(1, config.batch_size, config.hidden_size, requires_grad=True).to(device)
                cell_a    = torch.zeros(1, config.batch_size, config.hidden_size, requires_grad=True).to(device)

                # Forward
                output = model((hidden_k, cell_k), (hidden_a, cell_a), (k_train_mb.to(device),a_train_mb.to(device)))

                # Cost
                loss = criterion(output.to(device), y_train_mb.squeeze().to(device))

                wandb.log({"loss": loss})

                if (epoch) % 100 == 0 and batch_idx % 2 == 0:
                    print('Epoch {:3d}/{} Batch: {:2d} Cost: {:.6f}'.format(epoch, config.epochs, batch_idx, loss))
                
                # Backpropagate
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
        model.eval()

        for batch_idx, samples in enumerate(test_dataloader):
            k_train_mb, a_train_mb, y_train_mb = samples

            hidden_k    = torch.zeros(1, test_size, config.hidden_size).to(device)
            cell_k      = torch.zeros(1, test_size, config.hidden_size).to(device)
            hidden_a    = torch.zeros(1, test_size, config.hidden_size).to(device)
            cell_a      = torch.zeros(1, test_size, config.hidden_size).to(device)

            output = model((hidden_k, cell_k), (hidden_a, cell_a), (k_train_mb.to(device),a_train_mb.to(device)))
            prediction = output.argmax(dim=1)
            correct = prediction.eq(y_train_mb.view_as(prediction)).sum().item()
            print(correct/test_size)
            wandb.log({"accuracy": correct/test_size})

In [ ]:
wandb.agent(sweep_id, train)

In [ ]:
wandb.finish()